In [1]:
import codecs
import os
import pandas as pd
import re
from os import listdir
from os.path import isfile, join

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [9]:
number_of_families = 3920
input_dir = 'gSpan_s50_l3'
file_names = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
context = pd.DataFrame()
for file_name in file_names:
    if 'members' not in file_name:
        continue
    graph_number = file_name.split('.')[0].split('_')[2]
    membership = pd.Series([0] * number_of_families)
    with codecs.open(join(input_dir, file_name), 'r', encoding='utf-8') as f:
        for line in f:
            membership[int(line.strip('\n').strip('\r').strip('F'))] = 1
    context[graph_number] = membership
context.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3920 entries, 0 to 3919
Columns: 1847 entries, 0000 to 1846
dtypes: int64(1847)
memory usage: 55.2 MB


In [3]:
context['total_graph'] = context.sum(axis=1)
context.loc[context['total_graph'] > 0]

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).
  warnings.warn(msg, UserWarning)


,0000,0001,0002,0003,0004,0005,0006,0007,0008,0009,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,total_graph
0,1,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,44
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,52
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,838
4,1,1,0,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,845
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3898,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3900,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3901,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
context.drop(columns=['total_graph'], inplace=True)

In [10]:
context_file_name = 'context.csv'
context.to_csv(context_file_name, header=False, index=False)

In [11]:
os.system('java -jar AOCPosetBuilder.jar -i ' + context_file_name + ' -a HERMES -d posets/families_simplified.dot -f SIMPLIFIED -z')
#os.system('dot -Tpdf posets/families_simplified.dot -o posets/families_simplified.pdf')
#os.system('java -jar AOCPosetBuilder.jar -i ' + context_file_name + ' -a HERMES -d posets/families_full.dot -f FULL -z')

0

In [12]:
os.system('java -jar AOCPosetBuilder.jar -i ' + context_file_name + ' -a HERMES -d posets/families_minimal.dot -f MINIMAL')

0

Adding extent & intent size to the minimal version

In [8]:
file_min = codecs.open('posets/families_minimal.dot', 'w')
file_simp = 'posets/families_simplified.dot'
with codecs.open(file_simp) as f:
    for line in f:
        if 'graph' in line or 'rankdir' in line or '->' in line or line == '}':
            file_min.write(line)
            continue
        concept_id = re.search('<(.*)>', line).group(1)
        line_elements = line.split('|')
        line_elements[0] = line_elements[0].replace('> (', '> ' + concept_id + '|(').replace('I', 'Gr').replace('E', 'Fam')
        to_be_written = '|'.join(line_elements) + '\n'
        file_min.write(line_elements[0] + '}\"];\n')
file_min.close()
os.system('dot -Tpdf posets/families_minimal.dot -o posets/families_minimal.pdf')

0